In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [1]:
import warnings
warnings.simplefilter('ignore')

# Sparse Retriever(BM25)

## 前処理

In [2]:
# !pip install janome

In [3]:
from typing import List
import requests

# 日本語ストップワード辞書
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ja/master/stopwords-ja.txt"
stopwords_jp = requests.get(url).text.split("\n")

def remove_stopwords(tokens: List[str]) -> List[str]:
    """指定した単語リストからストップワードを除去した結果を返す"""
    tokens = [token for token in tokens if token not in stopwords_jp]
    return tokens

from janome.tokenizer import Tokenizer
def preprocess_jp(text: str) -> List[str]:
    """日本語テキストを前処理してトークンリストを返す"""

    # 改行コードの除去
    text = text.replace("\n", "")

    # Janomeで特定の品詞のみのトークンリストを作成
    pos_list = ["名詞", "動詞", "形容詞"]
    tokenizer = Tokenizer()
    tokens = tokenizer.tokenize(text)

    word_list = [token.surface for token in tokens if token.part_of_speech.split(',')[0] in pos_list]

    return remove_stopwords(word_list)

## BM25Retriever

In [4]:
# !pip install rank_bm25

In [ ]:
# def docs2text(docs):
#     texts=[]
#     for doc in docs:
#         text = doc.page_content.replace("\n","")
#         text = remove_space(text).replace("、 ","、").replace("。 ","。")
#         texts.append(text)
#     return texts

# texts = docs2text(docs)
# metadata = [doc.metadata for doc in docs]

In [5]:
from langchain.retrievers.bm25 import BM25Retriever
bm25_retriever = BM25Retriever.from_documents(docs,preprocess_func=preprocess_jp)
bm25_retriever.k = 5

In [ ]:
#　テスト
# bm25_retriever.get_relevant_documents("マッキンゼー社とは、について解説してください。")

In [ ]:
import pickle

# BM25Retrieverの保存
with open('pickled.pkl', 'wb') as f:
    pickle.dump(bm25_retriever, f)

# BM25Retrieverの読み込み
with open('pickled.pkl', 'rb') as f:
    bm25_retriever_ = pickle.load(f)  # 復元

## RetrievalQA

In [ ]:
# LLMの定義
llm = ...

In [6]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=bm25_retriever, 
    verbose=False
)

In [ ]:
qa.